In [14]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time
import matplotlib.pyplot as plt

XPATH = By.XPATH

# url = 'https://www.tinkoff.ru/invest/bonds/?start=0&end=12&country=Russian&orderType=Desc&sortType=ByYieldToClient'
url = 'https://www.tinkoff.ru/invest/bonds/?start=0&end=2400&country=Russian&orderType=Desc&sortType=ByYieldToClient'

In [97]:
class Obligations_parser:
    def __init__(self, driver, new_session = True):
        self.driver = driver
        self.driver.maximize_window()

        if new_session:
            self.driver.get(url)
            
            
        # self.obligations_data = pd.DataFrame(columns = ['Название', "Код", 'Ссылка', "Срок(месяца)", "Цена", "Номинал", "Выплат в год", "Величина купона(-13%)", "Прибыль в год(-13% с купона)", "Доходность(все время)", "Доходность(год)", "Амортизация", 'Рейтинг'])
        self.obligations_data = pd.DataFrame()
    

    def next_layer(self):
        next_page_class = 'Pagination-module__item_ALrvz Pagination-module__item_size_s_voSgR Pagination-module__item_arrow_mCpmW'
        # retry_counter = 0
        
        try:
            self.driver.find_element(XPATH, f'//a[@class = "{next_page_class}"]').click()
            self.elements_iterator()
            return True

        except Exception:
            print('Все страницы пройдены.')

            # self.elements_iterator()
            return(self.obligations_data)
        


    
    def grab_paper_info(self, paper_url : str = ''):
        if paper_url:
            self.driver.execute_script("window.open('');")
            self.driver.switch_to.window(self.driver.window_handles[-1])
            self.driver.get(paper_url)
        
        else:
            self.driver.get(paper_url)

        paper_data = {}

        try:
            paper_data.update({'Название' : [self.driver.find_element(XPATH, '//span[@class = "SecurityHeader__showName_iw6qC"]').text]})
            
            paper_data.update({'Код' : [self.driver.find_element(XPATH, '//span[@class = "SecurityHeader__ticker_j7fZW"]').text]})


            paper_time = self.driver.find_element(XPATH, '//div[@class = "SecurityHeader__panelText_KDJdO"]').text
            paper_time = paper_time[paper_time.index('з') + 3 : ]
            paper_data.update({'Срок(месяца)' : [int(paper_time[ : paper_time.index(' ')])]})
            del paper_time

            paper_data.update({'Цена' : [float(self.driver.find_element(XPATH, '//div[@class = "SecurityInvitingScreen__price_FSP8P"]').text.replace('\n', '').replace(',', '.').replace(' ', '')[ : -1])]})
            
            paper_data.update({'Рейтинг' : [self.driver.find_element(XPATH, '//div[@class = "SecurityHeader__panel_itBzT SecurityHeader__desktop_dL7RD"]/div[@class = "SecurityHeader__panelText_KDJdO"]').text]})

            paper_datas = self.driver.find_elements(XPATH, '//td[@class = "Table-module__cell_RJ0qz"]')



            for counter in range(1, len(paper_datas) + 1, 2):
                match paper_datas[counter - 1].text:
                    case "Величина купона":
                        paper_coupon = float(paper_datas[counter].text.replace('\n', '').replace(',', '.').replace(' ', '')[ : -2])
                        paper_coupon -= paper_coupon * 0.13     # Вычитаем налог 13% на купоны

                        paper_data.update({'Величина купона(-13%)' : [round(paper_coupon, 2)]})
                    
                    case "Номинал":
                        paper_data.update({'Номинал' : [round(float(paper_datas[counter].text.replace('\n', '').replace(',', '.').replace(' ', '')[ : -1]), 2)]})
                    
                    case "Количество выплат в год":
                        if 'Величина купона(-13%)' not in paper_data:
                            paper_data.update({'Величина купона(-13%)' : [0]})
                            
                        paper_data.update({'Выплат в год' : [int(paper_datas[counter].text)]})
                        paper_data.update({'Доходность(год)' : [round(paper_data['Выплат в год'][0] * paper_data['Величина купона(-13%)'][0], 2)]})
                        # paper_data.update({'Доходность(все время)' : [paper_data['Срок(месяца)'][0] // 2 * paper_data['Доходность(год)'][0]]})
                        # paper_data.update({'Доходность(все время)' : [paper_data['Срок(месяца)'][0] // paper_data['Выплат в год'][0] * paper_data['Величина купона(-13%)'][0]]})

                        paper_data.update({'Доходность(все время)' : [round(paper_data['Срок(месяца)'][0] * 12 / paper_data['Выплат в год'][0] * paper_data['Величина купона(-13%)'][0], 2)]})
                        paper_data.update({'Доходность(% в год)' : round(paper_data['Доходность(год)'][0] / paper_data['Номинал'][0] * 100, 2)})

                    
                    case 'Амортизация':
                        paper_data.update({'Амортизация' : [True] if paper_datas[counter] == 'Да' else [False]})
                    

            print(paper_data)

            paper_data.update({'Ссылка' : [paper_url]})

            

            # print(pd.DataFrame(paper_data))
            # print(pd.DataFrame.from_dict(paper_data), )

            

            self.obligations_data = pd.concat([self.obligations_data, pd.DataFrame(paper_data)])

            
            self.driver.close()

        except Exception:
            self.driver.close()
            time.sleep(10)
            self.grab_paper_info()

    def elements_iterator(self):

        link_buttons_class = 'Link-module__link_yQVl1 Link-module__link_theme_default_mkRhf'

        # Использую тр и тд, поскольку на сайте в строках тр 5 ссылок с нужным адресом, потому просто по классу ссылки не выйдет найти одно совпадение - придется потом чистить список. Приходится потихоьнку находить элементы.
        tr_link_buttons_class = 'Table-module__row_Qlwsh Table-module__row_clickable_FeO1O'
        # td_link_buttons_class = 'Table-module__cell_RJ0qz'

        papers_rows = self.driver.find_elements(XPATH, f'//tr[@class = "{tr_link_buttons_class}"]')
        
        for paper in papers_rows:
            self.driver.switch_to.window(self.driver.window_handles[-1])

            paper_link = paper.find_element(XPATH, 'td').find_element(XPATH, 'a').get_attribute('href')

            
            self.grab_paper_info(paper_link)
            print(len(self.obligations_data), '\n')

            

    
        return(self.obligations_data)

            

        # self.next_layer()



        # for paper in papers_rows:
        #     paper = paper.find_element(XPATH, '//td').find_element(XPATH, f'//a[@class = "{link_buttons_class}"]')
        #     print(paper.get_attribute('href'))




In [98]:
a = webdriver.Chrome()

In [99]:
parser_object = Obligations_parser(a, True)

In [100]:
parser_object.elements_iterator()

{'Название': ['Ferroni выпуск 1'], 'Код': ['RU000A103XP8'], 'Срок(месяца)': [24], 'Цена': [745.9], 'Рейтинг': ['Низкий'], 'Величина купона(-13%)': [9.22], 'Номинал': [1000.0], 'Выплат в год': [12], 'Доходность(год)': [110.64], 'Доходность(все время)': [221.28], 'Доходность(% в год)': 11.06}
1 

{'Название': ['Феррони БО-01'], 'Код': ['RU000A1053R3'], 'Срок(месяца)': [22], 'Цена': [906.6], 'Рейтинг': ['Низкий'], 'Величина купона(-13%)': [11.74], 'Номинал': [1000.0], 'Выплат в год': [12], 'Доходность(год)': [140.88], 'Доходность(все время)': [258.28], 'Доходность(% в год)': 14.09}
2 

{'Название': ['Роснано выпуск 8'], 'Код': ['RU000A1008V9'], 'Срок(месяца)': [53], 'Цена': [865.0], 'Рейтинг': ['Средний'], 'Величина купона(-13%)': [51.33], 'Номинал': [1000.0], 'Выплат в год': [2], 'Доходность(год)': [102.66], 'Доходность(все время)': [16322.94], 'Доходность(% в год)': 10.27}
3 

{'Название': ['ХК Финанс выпуск 8'], 'Код': ['RU000A104XU6'], 'Срок(месяца)': [6], 'Цена': [997.2], 'Рейтинг': 

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=117.0.5938.132)
Stacktrace:
	GetHandleVerifier [0x00007FF67E537892+54818]
	(No symbol) [0x00007FF67E4A6AC2]
	(No symbol) [0x00007FF67E35DA3B]
	(No symbol) [0x00007FF67E33DEE8]
	(No symbol) [0x00007FF67E3C3237]
	(No symbol) [0x00007FF67E3D6C6F]
	(No symbol) [0x00007FF67E3BE883]
	(No symbol) [0x00007FF67E393691]
	(No symbol) [0x00007FF67E3948D4]
	GetHandleVerifier [0x00007FF67E89B992+3610402]
	GetHandleVerifier [0x00007FF67E8F1860+3962352]
	GetHandleVerifier [0x00007FF67E8E9D4F+3930847]
	GetHandleVerifier [0x00007FF67E5D3646+693206]
	(No symbol) [0x00007FF67E4B1628]
	(No symbol) [0x00007FF67E4AD934]
	(No symbol) [0x00007FF67E4ADA62]
	(No symbol) [0x00007FF67E49E113]
	BaseThreadInitThunk [0x00007FF9F1477344+20]
	RtlUserThreadStart [0x00007FF9F1D426B1+33]


Ошибка     
49 paper_time = self.driver.find_element(XPATH, '//div[@class = "SecurityHeader__panelText_KDJdO"]').text
---> 50 paper_time = paper_time[paper_time.index('з') + 3 : ]
     51 paper_data.update({'Срок(месяца)' : [int(paper_time[ : paper_time.index(' ')])]})

ValueError: substring not found

на https://www.tinkoff.ru/invest/bonds/RU000A106NT4/

In [101]:
df = parser_object.obligations_data.reset_index().drop('index', axis = 1)

In [106]:
df.sort_values(['Доходность(% в год)'], ascending=False)

,Название,Код,Срок(месяца),Цена,Рейтинг,Величина купона(-13%),Номинал,Выплат в год,Доходность(год),Доходность(все время),Доходность(% в год),Ссылка
725,АПРИ Флай Плэнинг,RU000A105DS9,37,1014.900,Средний,52.03,1000.0,4.0,208.12,5775.33,20.81,https://www.tinkoff.ru/invest/bonds/RU000A105DS9/
703,Центр-Резерв БО-01,RU000A104TG3,24,1111.215,Низкий,14.88,950.0,12.0,178.56,357.12,18.80,https://www.tinkoff.ru/invest/bonds/RU000A104TG3/
18,АПРИ Флай Плэнинг выпуск 3,RU000A106WZ2,47,1021.500,Средний,45.50,1000.0,4.0,182.00,6415.50,18.20,https://www.tinkoff.ru/invest/bonds/RU000A106WZ2/
170,Завод КЭС выпуск 3,RU000A105AW7,36,1110.700,Средний,43.33,1000.0,4.0,173.32,4679.64,17.33,https://www.tinkoff.ru/invest/bonds/RU000A105AW7/
724,Глобал Факторинг Нетворк Рус,RU000A105JN7,25,1129.900,Низкий,14.27,1000.0,12.0,171.24,356.75,17.12,https://www.tinkoff.ru/invest/bonds/RU000A105JN7/
...,...,...,...,...,...,...,...,...,...,...,...,...
522,ПКТ выпуск 2,RU000A0JW5E3,28,751.100,Высокий,0.00,1000.0,2.0,0.00,0.00,0.00,https://www.tinkoff.ru/invest/bonds/RU000A0JW5E3/
783,СОПФ Инфраструктурные облигации выпуск 6,RU000A106P97,46,1006.800,Высокий,0.00,1000.0,4.0,0.00,0.00,0.00,https://www.tinkoff.ru/invest/bonds/RU000A106P97/
581,РУСАЛ Братск выпуск 1,RU000A0JWDN6,30,748.200,Высокий,0.00,1000.0,2.0,0.00,0.00,0.00,https://www.tinkoff.ru/invest/bonds/RU000A0JWDN6/
784,РЕСО-Лизинг выпуск 6,RU000A1049Y7,26,1020.000,Средний,0.00,1000.0,2.0,0.00,0.00,0.00,https://www.tinkoff.ru/invest/bonds/RU000A1049Y7/


In [103]:
if df['Код'].size >= 800:
    df.to_csv('Obligations_data_parsed.csv')

In [31]:
df[df['Номинал'] <= 15000].sort_values(by=['Доходность(год)'], ascending=False)

,Название,Код,Срок(месяца),Цена,Рейтинг,Величина купона(-13%),Номинал,Выплат в год,Доходность(год),Доходность(все время),Ссылка
290,ПЮДМ БО-П2,RU000A1020K7,21,10037.000000,Средний,92.916,10000.00,12.0,1114.992,1951.236,https://www.tinkoff.ru/invest/bonds/RU000A1020K7/
533,Нафтатранс Плюс выпуск 1,RU000A100303,3,10012.000000,Средний,84.303,10000.00,12.0,1011.636,252.909,https://www.tinkoff.ru/invest/bonds/RU000A100303/
678,Нафтатранс Плюс выпуск 2,RU000A100YD8,11,4664.466400,Низкий,49.590,4664.00,12.0,595.080,545.490,https://www.tinkoff.ru/invest/bonds/RU000A100YD8/
800,Селигдар Золотые облигации,RU000A1062M5,54,5609.824379,Высокий,70.992,5913.17,4.0,283.968,11500.704,https://www.tinkoff.ru/invest/bonds/RU000A1062M5/
130,АПРИ Флай Плэнинг,RU000A105DS9,37,1009.700000,Средний,52.026,1000.00,4.0,208.104,5774.886,https://www.tinkoff.ru/invest/bonds/RU000A105DS9/
...,...,...,...,...,...,...,...,...,...,...,...
806,ЕАБР выпуск 3Р-007,RU000A1064L3,54,1010.000000,Средний,0.000,1000.00,2.0,0.000,0.000,https://www.tinkoff.ru/invest/bonds/RU000A1064L3/
779,СОПФ Инфраструктурные облигации выпуск 6,RU000A106P97,46,1006.900000,Высокий,0.000,1000.00,4.0,0.000,0.000,https://www.tinkoff.ru/invest/bonds/RU000A106P97/
777,ОФЗ 29015,SU29015RMFS3,60,992.010000,Высокий,0.000,1000.00,4.0,0.000,0.000,https://www.tinkoff.ru/invest/bonds/SU29015RMFS3/
805,Норильский никель БО-09,RU000A1069N8,55,1009.500000,Высокий,0.000,1000.00,4.0,0.000,0.000,https://www.tinkoff.ru/invest/bonds/RU000A1069N8/


In [ ]:
df[df['Код'] == 'RU000A106WZ2']

,Название,Код,Срок(месяца),Цена,Рейтинг,Величина купона(-13%),Номинал,Выплат в год,Доходность(год),Доходность(все время),Ссылка
15,АПРИ Флай Плэнинг выпуск 3,RU000A106WZ2,47,1021.1,Средний,45.501,1000.0,4.0,182.004,6415.641,https://www.tinkoff.ru/invest/bonds/RU000A106WZ2/


In [ ]:
df_sliced = df[['Доходность(год)', 'Номинал', 'Код']][df['Номинал'] <= 15000]
df_sliced

,Доходность(год),Номинал,Код
0,110.664,1000.0,RU000A103XP8
1,140.940,1000.0,RU000A1053R3
2,123.540,1000.0,RU000A106FN3
3,84.564,1000.0,RU000A102S23
5,140.940,1000.0,RU000A105P64
...,...,...,...
809,112.752,1000.0,RU000A106K43
810,68.904,1000.0,RU000A103RS4
811,158.688,1000.0,RU000A104DZ7
812,153.468,1000.0,RU000A105ZF9


In [36]:
df_sliced_2k = df_sliced[df_sliced['Номинал'] <= 2000]

In [87]:
def plot_papers(df : pd.DataFrame):
    if 'Доходность(все время)' in df.columns:
        target_column_name = 'Доходность(все время)'

    elif 'Доходность(год)' in df.columns:
        target_column_name = 'Доходность(год)'
    
    else:
        target_column_name = None

    
    plt.figure(figsize=(20, 8))
    plt.scatter(df[target_column_name], df['Номинал'], c='b', marker='o', s=10)

    # Добавляем аннотации для каждой точки
    for i, row in df.iterrows():
        plt.annotate(row['Код'], (row[target_column_name], row['Номинал']), textcoords="offset points", xytext=(0, 15), ha='center', )

    # Настройки осей и заголовка
    plt.xlabel('Номинал')
    plt.ylabel(target_column_name)
    plt.title('Зависимость цены от доходности для бумаг')
    # Отображаем график
    plt.grid()
    plt.legend()
    plt.show()


In [92]:
import pandas as pd
import plotly.express as px

def plot_papers(df: pd.DataFrame):
    if 'Доходность(все время)' in df.columns:
        target_column_name = 'Доходность(все время)'
    elif 'Доходность(год)' in df.columns:
        target_column_name = 'Доходность(год)'
    else:
        target_column_name = None

    # Используйте Plotly Express для создания интерактивного графика
    fig = px.scatter(df, x=target_column_name, y='Номинал', color='Код', size_max=10, text='Код')

    # Настройки осей и заголовка
    fig.update_yaxes(title_text='Номинал')
    fig.update_xaxes(title_text=target_column_name)
    fig.update_layout(title='Зависимость цены от доходности для бумаг')

    # Отображаем график
    fig.show()

In [88]:
# df[df['Код'] == 'RU000A105DS9']
# df[df['Код'] == 'RU000A106WZ2']
# df[df['Код'] == 'RU000A105AW7']

In [ ]:
plot_papers(df_sliced[df_sliced['Номинал'] >= 1000])